In [1]:
import pandas as pd
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import Descriptors
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn import neural_network as nn
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
import numpy as np
import matplotlib.pyplot as plt

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = pd.read_csv("Permuted Data")
df

,Standardized_SMILES,pchembl_value,MolWt,LogP,QED
0,Brc1cncc(COCC2(c3ccccc3)CCNCC2)c1,4.82,361.283,3.6821,0.882288
1,C1=NC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCCCC...,6.70,397.526,6.4497,0.416926
2,C1=NC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCCCC...,6.47,398.514,4.8690,0.460447
3,C1=NC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCOCC...,5.39,399.498,5.2960,0.490800
4,C1=NCCC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCC...,7.05,397.526,6.1021,0.438245
...,...,...,...,...,...
1257,c1ccc2c3c([nH]c2c1)C(C1CCOCC1)NC(c1nc(-c2cscn2...,5.36,405.527,4.3692,0.472923
1258,c1ccc2c3c([nH]c2c1)C(C1CCOCC1)NC(c1nc(C24CC5CC...,6.30,456.634,5.7135,0.472659
1259,c1ccc2c3c([nH]c2c1)C(C1CCOCC1)NC(c1nc(C2CCCCC2...,6.87,404.558,5.2934,0.555363
1260,c1cncc(-c2c[nH]c(C3Cc4c([nH]c5ccccc45)C(C4CCCC...,6.72,397.526,5.4614,0.429771


In [16]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt


# Create a DeepChem dataset
tasks = ["pchembl_value"]
featurizer = dc.feat.CircularFingerprint(size=2048)
loader = dc.data.CSVLoader(tasks=tasks, feature_field="Standardized_SMILES", featurizer=featurizer)
dataset = loader.create_dataset("Permuted Data")

# Define the scaffold splitter
splitter = dc.splits.RandomSplitter()

# Get scaffold split indices
train_valid_test = splitter.k_fold_split(dataset, k=10, seed=42)


In [17]:
#MLP

import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = nn.MLPRegressor(hidden_layer_sizes=(min(25,round(np.sqrt(X_train.shape[0])),)), random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1136, Test size: 126


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 1 Accuracy: (0.8174286635650406, 1.7194375988472945e-31)
Fold 1 rmse: 0.5918019158800921
Fold 1 r_squared: 0.6056697815321024
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.7753327381323353
Fold 2 - Train size: 1136, Test size: 126


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 2 Accuracy: (0.8337074633228073, 8.962203322388869e-34)
Fold 2 rmse: 0.6136909276219665
Fold 2 r_squared: 0.6630132961156276
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.7820835882489195
Fold 3 - Train size: 1136, Test size: 126


...

Fold 8 Accuracy: (0.6903368413186128, 3.811342491651202e-19)
Fold 8 rmse: 0.7715918267556403
Fold 8 r_squared: 0.3826190196654946
Fold 8 Mean of Two Highest Tanimoto Similarities: 0.7636940590927096
Fold 9 - Train size: 1135, Test size: 127


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 9 Accuracy: (0.753829653264373, 1.4935813066910657e-24)
Fold 9 rmse: 0.7149865152889946
Fold 9 r_squared: 0.5449624290930057
Fold 9 Mean of Two Highest Tanimoto Similarities: 0.7536947454114042
Fold 10 - Train size: 1135, Test size: 127


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 10 Accuracy: (0.8254303761381461, 7.811288463830499e-33)
Fold 10 rmse: 0.5875297852558727
Fold 10 r_squared: 0.6371546597725494
Fold 10 Mean of Two Highest Tanimoto Similarities: 0.7768641862163611
Mean Pearson R: 0.3948983329929679
Overall Mean of Two Highest Tanimoto Similarities: 0.7724786926212768


In [18]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("cv_results_MLP_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,COc1ccc2ncc(F)c(CC(=O)C34CCC(NCc5ccc6c(n5)NC(=...,5.03,5.023331
1,1,Cc1cccnc1CN1CCC2(CC1)C(=O)N(c1ccc(-c3ccc(C(=O)...,4.72,4.748082
2,1,CCc1nc2ccc3c(c2s1)CCN(CCCSc1nnc(-c2cccc4nc(C)c...,6.80,6.192729
3,1,COc1ccc2ncc(F)c(C(CO)CC34CCC(NCc5ccc6c(n5)NC(=...,4.96,4.957776
4,1,COc1cc(N2C(=O)N(c3ccc(-c4ccc(C(=O)N(C)C)cc4)cc...,5.05,5.235861
...,...,...,...,...
11353,10,O=C1N(CCN2CCCC(OCc3ccccc3)C2)CCN1c1cccc(Cl)c1,7.10,6.954194
11354,10,O=C1COc2ccc(CNC34CCC(CCc5c(F)cnc6cc(C(F)(F)F)c...,9.05,9.199877
11355,10,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccccc3C(F)(F)F)C...,5.20,5.305914
11356,10,O=C(Nc1ccc(Cl)c(Cl)c1)N1CCN(C(=O)C2CN(C3CC3)CC...,4.72,4.700367


In [19]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("cv_results_MLP_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,O=C1N(CCN2Cc3ccccc3C2)CCN1Cc1cccc(C(F)(F)F)c1,5.20,5.386571
1,1,Cc1oc(-c2ccccc2)nc1C(=O)N1CCC(Oc2ccc(CN3CCCC3)...,5.52,5.731040
2,1,COc1ccc(-c2nnc(C(=O)N3CC(Oc4ccc(CN5CCC(C)(O)C5...,4.58,4.575710
3,1,CNC(C)c1cc(C(F)(F)F)ccc1Oc1ccc(Cl)c(Cl)c1,5.52,5.575488
4,1,COCCN(CCc1ccc(Cl)cc1)C1CCN(c2nc(=N)[nH][nH]2)CC1,5.55,4.832247
...,...,...,...,...
1257,10,COc1cc(N2CCN(CCO)CC2)ccc1N=c1nc(-c2cnc3ccccn23...,5.43,5.663160
1258,10,Cc1nnc(C2(c3cnn(C)c3)NC(c3nc(-c4ccc(F)cn4)c[nH...,5.50,4.322862
1259,10,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccc(C(C)(C)C)cc3...,6.20,6.007072
1260,10,NC(=O)c1cccc(NC2CC3CCC(C2)N3Cc2ccccc2)c1,5.96,6.406314


In [20]:
#RF

In [21]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = RandomForestRegressor(n_estimators=min(25,round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1136, Test size: 126
Fold 1 Accuracy: (0.7895075119797272, 4.624729234625119e-28)
Fold 1 rmse: 0.588950400261562
Fold 1 r_squared: 0.6094606776762858
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.7753327381323353
Fold 2 - Train size: 1136, Test size: 126
Fold 2 Accuracy: (0.8858433011001012, 3.5685389787045907e-43)
Fold 2 rmse: 0.492308721375985
Fold 2 r_squared: 0.7831355232838868
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.7820835882489195
Fold 3 - Train size: 1136, Test size: 126
Fold 3 Accuracy: (0.8692223024625326, 9.598033817109418e-40)
Fold 3 rmse: 0.4511022190850894
Fold 3 r_squared: 0.7539846867160431
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.7809259161776553
Fold 4 - Train size: 1136, Test size: 126
Fold 4 Accuracy: (0.870042655017261, 6.670503237547208e-40)
Fold 4 rmse: 0.4944296672662512
Fold 4 r_squared: 0.7530293918181792
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.7711189457275879
Fold 5 - Train size: 1136, Test size: 

In [22]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("cv_results_RF_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,COc1ccc2ncc(F)c(CC(=O)C34CCC(NCc5ccc6c(n5)NC(=...,5.03,5.0714
1,1,Cc1cccnc1CN1CCC2(CC1)C(=O)N(c1ccc(-c3ccc(C(=O)...,4.72,4.7260
2,1,CCc1nc2ccc3c(c2s1)CCN(CCCSc1nnc(-c2cccc4nc(C)c...,6.80,6.2600
3,1,COc1ccc2ncc(F)c(C(CO)CC34CCC(NCc5ccc6c(n5)NC(=...,4.96,5.3390
4,1,COc1cc(N2C(=O)N(c3ccc(-c4ccc(C(=O)N(C)C)cc4)cc...,5.05,5.0052
...,...,...,...,...
11353,10,O=C1N(CCN2CCCC(OCc3ccccc3)C2)CCN1c1cccc(Cl)c1,7.10,6.5936
11354,10,O=C1COc2ccc(CNC34CCC(CCc5c(F)cnc6cc(C(F)(F)F)c...,9.05,8.7748
11355,10,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccccc3C(F)(F)F)C...,5.20,5.4040
11356,10,O=C(Nc1ccc(Cl)c(Cl)c1)N1CCN(C(=O)C2CN(C3CC3)CC...,4.72,4.7876


In [23]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("cv_results_RF_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,O=C1N(CCN2Cc3ccccc3C2)CCN1Cc1cccc(C(F)(F)F)c1,5.20,5.7272
1,1,Cc1oc(-c2ccccc2)nc1C(=O)N1CCC(Oc2ccc(CN3CCCC3)...,5.52,5.0024
2,1,COc1ccc(-c2nnc(C(=O)N3CC(Oc4ccc(CN5CCC(C)(O)C5...,4.58,4.9432
3,1,CNC(C)c1cc(C(F)(F)F)ccc1Oc1ccc(Cl)c(Cl)c1,5.52,5.5756
4,1,COCCN(CCc1ccc(Cl)cc1)C1CCN(c2nc(=N)[nH][nH]2)CC1,5.55,5.1520
...,...,...,...,...
1257,10,COc1cc(N2CCN(CCO)CC2)ccc1N=c1nc(-c2cnc3ccccn23...,5.43,5.1562
1258,10,Cc1nnc(C2(c3cnn(C)c3)NC(c3nc(-c4ccc(F)cn4)c[nH...,5.50,4.9346
1259,10,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccc(C(C)(C)C)cc3...,6.20,6.1452
1260,10,NC(=O)c1cccc(NC2CC3CCC(C2)N3Cc2ccccc2)c1,5.96,5.9392


In [24]:
#xgboost

In [31]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = xgb.XGBRegressor(n_estimators=min(25, round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1136, Test size: 126
Fold 1 Accuracy: (0.8012211200686475, 1.9625015395159476e-29)
Fold 1 rmse: 0.5662359338906666
Fold 1 r_squared: 0.6390041751007192
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.7753327381323353
Fold 2 - Train size: 1136, Test size: 126
Fold 2 Accuracy: (0.9024005655307712, 3.6360213930869467e-47)
Fold 2 rmse: 0.4572634296678987
Fold 2 r_squared: 0.8129118422603518
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.7820835882489195
Fold 3 - Train size: 1136, Test size: 126
Fold 3 Accuracy: (0.8330431758666648, 1.1229096871679511e-33)
Fold 3 rmse: 0.5063717344208198
Fold 3 r_squared: 0.6900075442487212
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.7809259161776553
Fold 4 - Train size: 1136, Test size: 126
Fold 4 Accuracy: (0.8824759933667607, 1.9442455399598875e-42)
Fold 4 rmse: 0.4878837654840431
Fold 4 r_squared: 0.7595255380881518
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.7711189457275879
Fold 5 - Train size: 1136, Test 

In [32]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("cv_results_xgboost_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,COc1ccc2ncc(F)c(CC(=O)C34CCC(NCc5ccc6c(n5)NC(=...,5.03,5.333500
1,1,Cc1cccnc1CN1CCC2(CC1)C(=O)N(c1ccc(-c3ccc(C(=O)...,4.72,4.905906
2,1,CCc1nc2ccc3c(c2s1)CCN(CCCSc1nnc(-c2cccc4nc(C)c...,6.80,6.018429
3,1,COc1ccc2ncc(F)c(C(CO)CC34CCC(NCc5ccc6c(n5)NC(=...,4.96,5.394192
4,1,COc1cc(N2C(=O)N(c3ccc(-c4ccc(C(=O)N(C)C)cc4)cc...,5.05,5.186042
...,...,...,...,...
11353,10,O=C1N(CCN2CCCC(OCc3ccccc3)C2)CCN1c1cccc(Cl)c1,7.10,5.966371
11354,10,O=C1COc2ccc(CNC34CCC(CCc5c(F)cnc6cc(C(F)(F)F)c...,9.05,8.409837
11355,10,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccccc3C(F)(F)F)C...,5.20,5.763311
11356,10,O=C(Nc1ccc(Cl)c(Cl)c1)N1CCN(C(=O)C2CN(C3CC3)CC...,4.72,4.940247


In [33]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("cv_results_xgboost_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,O=C1N(CCN2Cc3ccccc3C2)CCN1Cc1cccc(C(F)(F)F)c1,5.20,5.707836
1,1,Cc1oc(-c2ccccc2)nc1C(=O)N1CCC(Oc2ccc(CN3CCCC3)...,5.52,5.221400
2,1,COc1ccc(-c2nnc(C(=O)N3CC(Oc4ccc(CN5CCC(C)(O)C5...,4.58,4.921556
3,1,CNC(C)c1cc(C(F)(F)F)ccc1Oc1ccc(Cl)c(Cl)c1,5.52,5.603234
4,1,COCCN(CCc1ccc(Cl)cc1)C1CCN(c2nc(=N)[nH][nH]2)CC1,5.55,5.634065
...,...,...,...,...
1257,10,COc1cc(N2CCN(CCO)CC2)ccc1N=c1nc(-c2cnc3ccccn23...,5.43,5.274371
1258,10,Cc1nnc(C2(c3cnn(C)c3)NC(c3nc(-c4ccc(F)cn4)c[nH...,5.50,5.243750
1259,10,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccc(C(C)(C)C)cc3...,6.20,5.986852
1260,10,NC(=O)c1cccc(NC2CC3CCC(C2)N3Cc2ccccc2)c1,5.96,6.700325
